<a href="https://colab.research.google.com/github/MATan1808/-n-t-p-SQL/blob/main/Navie_bean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
filepath='/content/sample_data/Dry_Bean_Dataset.xlsx'

In [ ]:

# Load data
data = pd.read_excel(filepath)

In [ ]:
# Print the column names
print(data.columns)

Index(['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
       'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent',
       'Solidity', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2',
       'ShapeFactor3', 'ShapeFactor4', 'Class'],
      dtype='object')


In [ ]:
# Kiểm tra thông tin cơ bản về tập dữ liệu
info = data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13611 entries, 0 to 13610
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             13611 non-null  int64  
 1   Perimeter        13611 non-null  float64
 2   MajorAxisLength  13611 non-null  float64
 3   MinorAxisLength  13611 non-null  float64
 4   AspectRation     13611 non-null  float64
 5   Eccentricity     13611 non-null  float64
 6   ConvexArea       13611 non-null  int64  
 7   EquivDiameter    13611 non-null  float64
 8   Extent           13611 non-null  float64
 9   Solidity         13611 non-null  float64
 10  roundness        13611 non-null  float64
 11  Compactness      13611 non-null  float64
 12  ShapeFactor1     13611 non-null  float64
 13  ShapeFactor2     13611 non-null  float64
 14  ShapeFactor3     13611 non-null  float64
 15  ShapeFactor4     13611 non-null  float64
 16  Class            13611 non-null  object 
dtypes: float64(1

In [ ]:
# Kiểm tra xem có dữ liệu nào bị thiếu không
missing_values = data.isnull().sum()

info, missing_values

(None,
 Area               0
 Perimeter          0
 MajorAxisLength    0
 MinorAxisLength    0
 AspectRation       0
 Eccentricity       0
 ConvexArea         0
 EquivDiameter      0
 Extent             0
 Solidity           0
 roundness          0
 Compactness        0
 ShapeFactor1       0
 ShapeFactor2       0
 ShapeFactor3       0
 ShapeFactor4       0
 Class              0
 dtype: int64)

In [ ]:
# Xác định các loại đậu khác nhau trong tập dữ liệu
unique_bean_classes = data['Class'].unique()
unique_bean_classes


array(['SEKER', 'BARBUNYA', 'BOMBAY', 'CALI', 'HOROZ', 'SIRA', 'DERMASON'],
      dtype=object)

# Phân Tích Thống Kê Cơ Bản

# Đặc điểm của các loại đậu

In [ ]:
# Tạo một từ điển để lưu trữ kết quả phân tích cho mỗi loại đậu
ket_qua_phat_tich_moi = {}

# Xác định các loại đậu khác nhau trong tập dữ liệu
cac_loai_dau_moi = data['Class'].unique()

# Lặp qua mỗi loại đậu và thực hiện phân tích
for loai_dau in cac_loai_dau_moi:
    # Lọc dữ liệu cho loại đậu cụ thể
    du_lieu_cu_the_moi = data[data['Class'] == loai_dau]

    # Tính toán thống kê cơ bản cho mỗi đặc trưng
    ket_qua_phat_tich_moi_dau = du_lieu_cu_the_moi.describe()

    # Lưu trữ kết quả vào từ điển
    ket_qua_phat_tich_moi[loai_dau] = ket_qua_phat_tich_moi_dau

# Chuẩn bị để hiển thị kết quả phân tích cho mỗi loại đậu
ket_qua_phat_tich_loai_moi = [(loai_dau, thong_ke) for loai_dau, thong_ke in ket_qua_phat_tich_moi.items()]
ket_qua_phat_tich_loai_moi[:2]  # Hiển thị kết quả phân tích cho hai loại đậu đầu tiên để ngắn gọn


[('SEKER',
                 Area    Perimeter  MajorAxisLength  MinorAxisLength  \
  count   2027.000000  2027.000000      2027.000000      2027.000000   
  mean   39881.299951   727.672440       251.291957       201.909653   
  std     4779.877395    47.849014        19.969504        10.942797   
  min    28395.000000   610.291000       200.524796       171.210559   
  25%    36408.000000   692.948500       237.037081       193.919169   
  50%    39180.000000   721.292000       249.152044       200.432810   
  75%    42700.000000   755.011500       262.802106       208.745418   
  max    61150.000000   933.372000       339.931533       252.287649   
  
         AspectRation  Eccentricity    ConvexArea  EquivDiameter       Extent  \
  count   2027.000000   2027.000000   2027.000000    2027.000000  2027.000000   
  mean       1.245182      0.584781  40269.567341     224.948441     0.771674   
  std        0.082223      0.071265   4823.866621      13.294146     0.018828   
  min        1

dựa vào đặc điểm của từng loại đậu sẽ ss lại những loại đậu trong nhãn dán
ARBUNYA: Loại đậu này có diện tích (Area) trung bình là khoảng 69804, với độ lệch chuẩn không quá cao, cho thấy sự ổn định trong kích thước. Chu vi (Perimeter) và các đặc trưng liên quan cũng có giá trị tương ứng. Điều này có thể giúp phân biệt BARBUNYA với các loại đậu khác.

BOMBAY: BOMBAY có diện tích và chu vi thấp hơn so với BARBUNYA. Đặc biệt, diện tích (Area) thấp hơn và chu vi (Perimeter) cũng thấp hơn. Điều này làm cho BOMBAY có hình dáng tròn hơn so với BARBUNYA.

CALI: CALI có diện tích (Area) cao hơn so với BARBUNYA và BOMBAY. Điều này cho thấy CALI thường có kích thước lớn hơn. Chu vi (Perimeter) và các đặc trưng khác cũng cao hơn.

DERMASON: DERMASON có diện tích và chu vi lớn nhất trong số tất cả các loại đậu. Điều này cho thấy DERMASON có hình dáng lớn và phức tạp hơn so với các loại đậu khác.

HOROZ: HOROZ có diện tích và chu vi trung bình, tương đối giống với BARBUNYA. Tuy nhiên, các đặc trưng khác có thể khác biệt và cần được xem xét.

SEKER: SEKER có diện tích (Area) thấp hơn so với BARBUNYA và BOMBAY, nhưng cao hơn HOROZ. Chu vi (Perimeter) và các đặc trưng khác có thể giúp phân biệt SEKER với các loại đậu khác.

SIRA: SIRA có diện tích và chu vi lớn, tương tự như CALI và DERMASON. Điều này có thể gây hiểu nhầm giữa các loại đậu lớn.

**Tổng kết các đặc điểm của từng loại đậu :**

BARBUNYA: Đậu này có kích thước trung bình, không quá lớn hoặc nhỏ. Diện tích và chu vi ổn định.

BOMBAY: BOMBAY có diện tích nhỏ hơn và hình dáng tròn hơn so với BARBUNYA.

CALI: CALI thường có kích thước lớn hơn, với diện tích và chu vi cao hơn so với BARBUNYA.

DERMASON: DERMASON là loại đậu có kích thước lớn nhất, với diện tích và chu vi cao. Đậu này có hình dáng phức tạp.

HOROZ: HOROZ có diện tích và chu vi tương đối trung bình, giống với BARBUNYA.

SEKER: SEKER có diện tích nhỏ hơn so với BARBUNYA và BOMBAY, nhưng cao hơn HOROZ.

SIRA: SIRA có diện tích và chu vi lớn, tương tự như CALI và DERMASON.

# **Navie Bayes**

**Đặc Trưng của các loại đậu**

In [ ]:
#Tính Tổng Giá Trị cho Tất Cả Đặc Trưng của Mỗi Loại Đậu
total_values_per_class = data.groupby('Class').sum()
print(total_values_per_class)


               Area    Perimeter  MajorAxisLength  MinorAxisLength  \
Class                                                                
BARBUNYA   92281064  1382951.820    489198.537357    317688.963461   
BOMBAY     90559201   827693.159    309625.382957    195412.029489   
CALI      123127284  1723943.879    667484.247679    385284.104492   
DERMASON  113892949  2358833.014    874292.111017    587420.229215   
HOROZ     103434325  1773489.455    718315.519261    355081.037994   
SEKER      80839395  1474992.036    509368.797550    409270.866258   
SIRA      117905983  2099359.792    789166.360045    502949.458468   

          AspectRation  Eccentricity  ConvexArea  EquivDiameter       Extent  \
Class                                                                          
BARBUNYA   2041.689750    997.666614    93896014  393045.165189   990.538497   
BOMBAY      827.657079    402.210508    91774447  244787.424526   405.363765   
CALI       2825.870743   1328.131100   125002260 

In [ ]:
#Tính giá trị trung bình cho tất cả các đặc trưng cho mỗi loại đậu trong dataset
mean_values_per_class = data.groupby('Class').mean()
mean_values_per_class

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
Class,,,,,,,,,,,,,,,,
BARBUNYA,69804.133132,1046.105764,370.044279,240.309352,1.544395,0.754665,71025.729198,297.311018,0.749273,0.982804,0.800200,0.805001,0.005357,0.001394,0.649144,0.995739
BOMBAY,173485.059387,1585.619079,593.152075,374.352547,1.585550,0.770518,175813.116858,468.941426,0.776559,0.986902,0.864421,0.792622,0.003442,0.000844,0.629195,0.991841
CALI,75538.211043,1057.634282,409.499538,236.370616,1.733663,0.814804,76688.503067,309.535280,0.758953,0.985021,0.845934,0.756703,0.005459,0.001107,0.573022,0.990584
DERMASON,32118.710942,665.209536,246.557279,165.657143,1.490471,0.736632,32498.435138,201.683813,0.752953,0.988226,0.908114,0.819110,0.007755,0.002161,0.671636,0.996914
HOROZ,53648.508817,919.859676,372.570290,184.170663,2.026119,0.867443,54440.091805,260.730715,0.706393,0.985480,0.794420,0.700880,0.007007,0.001048,0.491791,0.991926
SEKER,39881.299951,727.672440,251.291957,201.909653,1.245182,0.584781,40269.567341,224.948441,0.771674,0.990351,0.944508,0.896841,0.006334,0.002541,0.805149,0.998383
SIRA,44729.128604,796.418737,299.380258,190.800250,1.570083,0.767277,45273.099772,238.335316,0.749445,0.987971,0.884652,0.797345,0.006720,0.001683,0.636358,0.995385


**Tiến Hành Kiểm Tra**

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X = data.drop('Class', axis=1)  # Features
y = data['Class']  # Target variable

In [ ]:
# Tạo tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Xây dựng mô hình Naive Bayes
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [ ]:
# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

Tinh Chỉnh Mô Hình

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameters grid for GaussianNB (you can adjust this)
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid, cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Use the best model for predictions
best_model = grid_search.best_estimator_


In [ ]:
# Dự đoán trên tập kiểm tra sử dụng mô hình tốt nhất từ Grid Search
y_pred_best = best_model.predict(X_test)

# Đánh giá mô hình sau khi tinh chỉnh
accuracy_best = accuracy_score(y_test, y_pred_best)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)

print(f'Dộ chính xác sau khi tinh chỉnh: {accuracy_best}')



Dộ chính xác sau khi tinh chỉnh: 0.7579875137715755


In [ ]:
print('Confusion Matrix:')
print(conf_matrix_best)

Confusion Matrix:
[[120   0 108   0  25   0   8]
 [  0 117   0   0   0   0   0]
 [ 63   0 242   0  11   0   1]
 [  0   0   0 554   0  76  41]
 [ 14   0  20  11 325   0  38]
 [  3   0   0  78   5 285  42]
 [  0   0   0  26  43  46 421]]


In [ ]:
# Xác định nhãn gán sai
misclassified_labels = [i for i in range(len(y_test)) if y_test.iloc[i] != y_pred[i]]

In [ ]:
# In số liệu cụ thể và kết luận loại nào sai nhiều nhất
misclassified_counts = pd.Series(y_test.iloc[misclassified_labels]).value_counts()
most_misclassified_label = misclassified_counts.idxmax()


In [ ]:
print('\nSố liệu nhãn gán sai:')
print(misclassified_counts)
print(f'\nLoại gán sai nhiều nhất: {most_misclassified_label}')


Số liệu nhãn gán sai:
BARBUNYA    141
SEKER       128
DERMASON    117
SIRA        115
HOROZ        83
CALI         75
Name: Class, dtype: int64

Loại gán sai nhiều nhất: BARBUNYA


In [ ]:
# Xác định nhãn gán sai
misclassified = np.where(y_test != y_pred)[0]
num_misclassified = len(misclassified)

# num_misclassified
print("tổng số lượng hạt đâu sai là ", num_misclassified)

tổng số lượng hạt đâu sai là  659


In [ ]:
# Xử lý lại nhãn dán
data['Class'] = model.predict(X)


In [ ]:
# Kiểm tra đậu ngoại lai và xử lý
outlier_beans = data[~data['Class'].isin(unique_bean_classes)]
if len(outlier_beans) == 0:
    print('\nKhông có đậu ngoại lai.')
else:
    print(f'\nSố lượng đậu ngoại lai: {len(outlier_beans)}')
    # Xử lý đậu ngoại lai, ví dụ: xóa
    data = data[data['Class'].isin(unique_bean_classes)]


Không có đậu ngoại lai.


In [ ]:
# Đánh giá mô hình sau khi xử lý
X_new = data.drop('Class', axis=1)
y_new = data['Class']
y_pred_new = model.predict(X_new)
accuracy_after = accuracy_score(y_new, y_pred_new)
print(f'\nDộ chính xác sau khi xử lý: {accuracy_after}')



Dộ chính xác sau khi xử lý: 1.0


In [ ]:
# Đánh giá mô hình xác thực chéo
cross_val_scores = cross_val_score(model, X, y, cv=10)

print('\nĐánh giá mô hình xác thực chéo:')
print(cross_val_scores)
print(f'Mean Accuracy: {cross_val_scores.mean()}')


Đánh giá mô hình xác thực chéo:
[0.30396476 0.52681852 0.91623806 0.99338722 0.98897869 0.94562821
 0.92064658 0.87876561 0.58265981 0.18515797]
Mean Accuracy: 0.7242245433682799
